<a href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/HallucinationReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hallucination reduction using conditional RAG

In [ ]:
!pip install goodfire --quiet
!pip install datasets --quiet


In [ ]:
from google.colab import userdata

# Add your Goodfire API Key to your Colab secrets
GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')

## Initialize the SDK

In [ ]:
import goodfire

client = goodfire.Client(GOODFIRE_API_KEY)

# Instantiate a model variant
#variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")
variant = goodfire.Variant("meta-llama/Llama-3.3-70B-Instruct")

If we see the user is asking about something that might have insufficient info on some features, it will look for the document in RAG and complete it.

For example, if the user asks about Fed decisions, we can stop the request, get RAG data on Fed decisions and pass it back into the model.

In [ ]:
finance_features = client.features.search("financial fraud, market trends, investment risks", model=variant, top_k=10)
print(finance_features)

FeatureGroup([
   0: "Discussion of financial losses and investment risks",
   1: "Financial risk and investment caution",
   2: "Financial and business risk concepts",
   3: "Corporate fraud scandals and legal consequences",
   4: "Personal financial circumstances in investment advice",
   5: "The assistant should warn about investment risks",
   6: "Insider trading and market manipulation",
   7: "Financial market trading discussions and explanations",
   8: "Financial investment discussion and advice",
   9: "Price movements in financial market analysis"
])


In [ ]:
# Function to retrieve synthetic RAG data for finance
def get_rag_data(prompt):
    # This is where actual RAG retrieval would occur
    return (
        "According to Bloomberg and SEC filings, the stock market trends indicate volatility. "
        "Macroeconomic factors, inflation, and Federal Reserve policies significantly impact these trends. "
        "Always verify insights from sources such as Reuters, Bloomberg, or official financial statements."
    )

def generate_response(prompt):


    variant.reset()
    variant.abort_when(finance_features[0] > 0.25)  # This will not raise an error but will silently stop if triggered

    generated_tokens = ""

    try:
        # Generate initial response
        response = client.chat.completions.create(
            [
                {"role": "user", "content": prompt}
            ],
            model=variant,
            stream=True,
            max_completion_tokens=500,
        )

        for token in response:
            if token.choices[0].delta.content:
                generated_tokens += token.choices[0].delta.content

        # Print the initial generated response
        print("\n--- GENERATED RESPONSE ---\n")
        print(generated_tokens)

    except Exception as e:
        print(f"\nError: {e} - Falling back to RAG\n")

    # Always fall back to RAG if hallucination threshold was crossed
    rag_data = get_rag_data(prompt)
    print("\n--- NEW TOKENS (RAG-Verified Data) ---\n")

    variant.reset()

    response = client.chat.completions.create(
        [
            {"role": "system", "content": "You are a financial assistant providing fact-based insights. "
                                          "Use only verified data from Bloomberg, Reuters, or SEC filings. "
                                          "Here is the retrieved data: " + rag_data},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": generated_tokens},
        ],
        model=variant,
        stream=True,
        max_completion_tokens=500,
    )

    for token in response:
        if token.choices[0].delta.content:
            print(token.choices[0].delta.content, end="")

    return None

# **Finance Example Query to Check for Hallucination Handling**
generate_response("What are the expected Federal Reserve rate decisions for next quarter?")



--- GENERATED RESPONSE ---

The Federal Reserve's next meeting is scheduled for September. Based on current market expectations and economic data, here are the possible rate decision scenarios:

1. **No rate change**: The Fed might keep rates steady, given the recent inflation slowdown and stable economic growth.
2. **25-basis-point cut**: Some experts predict a small rate cut to support the economy and counterbalance global trade uncertainties.
3. **No changes to forward guidance**: The Fed might maintain its current forward guidance, indicating a neutral or slightly dovish stance.

Keep in mind that these are just market expectations and not official Fed announcements. The actual decision will depend on various factors, including inflation, employment, and global economic conditions.

Would you like me to provide more information or context about the Federal Reserve's decision-making process?

--- NEW TOKENS (RAG-Verified Data) ---

According to recent reports from Bloomberg and Reu